# Generic Nearest Neighborhood Analysis

In [ ]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
import time
import sys
import matplotlib.pyplot as plt
import math
import os

from sklearn.cluster import MiniBatchKMeans
import seaborn as sns
#import scanpy as sc

def get_windows(job,n_neighbors):
    start_time,idx,tissue_name,indices = job
    job_start = time.time()
    
    print ("Starting:", str(idx+1)+'/'+str(len(exps)),': ' + exps[idx])

    tissue = tissue_group.get_group(tissue_name)
    to_fit = tissue.loc[indices][[X,Y]].values

    fit = NearestNeighbors(n_neighbors=n_neighbors).fit(tissue[[X,Y]].values)
    m = fit.kneighbors(to_fit)

    #sort_neighbors
    args = m[0].argsort(axis = 1)
    add = np.arange(m[1].shape[0])*m[1].shape[1]
    sorted_indices = m[1].flatten()[args+add[:,None]]

    neighbors = tissue.index.values[sorted_indices]
   
    end_time = time.time()
   
    print ("Finishing:", str(idx+1)+"/"+str(len(exps)),": "+ exps[idx],end_time-job_start,end_time-start_time)
    return neighbors.astype(np.int32)


In [ ]:
#Choose this information and input based on above
ks = [100,200] # k=5 means it collects 5 nearest neighbors for each center cell
path_to_data = "G:/HuBMAP/B008_12/Neighborhoods/Individual_Neighborhoods/B004_12_Neighborhoods_Ind_combined.csv"

X = 'x'
Y = 'y'
reg = 'unique_region'
file_type = 'csv'

cluster_col = 'Neighborhood'
cellhier_path = 'G:/cellhier'
keep_cols = [X,Y,reg,cluster_col]
save_path = 'G:/HuBMAP/B008_12/Neighborhoods/'

# Data Preprocessing

##########################

In [ ]:
#Import Data
n_neighbors = max(ks)
sys.path.append(cellhier_path)
from cellhier.general import *

cells = pd.read_csv(path_to_data)
cells = pd.concat([cells,pd.get_dummies(cells[cluster_col])],1)
sum_cols = cells[cluster_col].unique()
values = cells[sum_cols].values

In [ ]:
#Get each region
tissue_group = cells[[X,Y,reg]].groupby(reg)
exps = list(cells[reg].unique())
tissue_chunks = [(time.time(),exps.index(t),t,a) for t,indices in tissue_group.groups.items() for a in np.array_split(indices,1)] 
tissues = [get_windows(job,n_neighbors) for job in tissue_chunks]

In [ ]:
#Loop over k to compute neighborhoods
out_dict = {}
for k in ks:
    for neighbors,job in zip(tissues,tissue_chunks):

        chunk = np.arange(len(neighbors))#indices
        tissue_name = job[2]
        indices = job[3]
        window = values[neighbors[chunk,:k].flatten()].reshape(len(chunk),k,len(sum_cols)).sum(axis = 1)
        out_dict[(tissue_name,k)] = (window.astype(np.float16),indices)
        
windows = {}
for k in ks:
   
    window = pd.concat([pd.DataFrame(out_dict[(exp,k)][0],index = out_dict[(exp,k)][1].astype(int),columns = sum_cols) for exp in exps],0)
    window = window.loc[cells.index.values]
    window = pd.concat([cells[keep_cols],window],1)
    windows[k] = window

In [ ]:
#Fill in based on above
k = 100
n_neighborhoods = 20
neighborhood_name = "neighborhood"+str(k)
k_centroids = {}

In [ ]:
#producing what to plot
windows2 = windows[k]
windows2[cluster_col] = cells[cluster_col]

km = MiniBatchKMeans(n_clusters = n_neighborhoods,random_state=0)

labels = km.fit_predict(windows2[sum_cols].values)
k_centroids[k] = km.cluster_centers_
cells[neighborhood_name] = labels

In [ ]:
#modify figure size aesthetics for each neighborhood
plt.rcParams["legend.markerscale"] = 5
figs = catplot(cells,X = 'Xcorr',Y='Ycorr',exp = 'array',
               hue = 'neighborhood'+str(k),invert_y=True,size = 1,figsize=8)

In [ ]:
neigh_list = [1,6,3]

In [ ]:
#this plot shows the types of cells (ClusterIDs) in the different niches (0-9)
k_to_plot = k
niche_clusters = (k_centroids[k_to_plot])
tissue_avgs = values.mean(axis = 0)
fc = np.log2(((niche_clusters+tissue_avgs)/(niche_clusters+tissue_avgs).sum(axis = 1, keepdims = True))/tissue_avgs)
fc = pd.DataFrame(fc,columns = sum_cols)
s=sns.clustermap(fc.iloc[neigh_list,:], vmin =-3,vmax = 3,cmap = 'bwr',figsize=(15,8))
#s.savefig(save_path+"celltypes_perniche_"+"_"+str(k)+".png", dpi=600)

In [ ]:
#modify figure size aesthetics for each neighborhood
plt.rcParams["legend.markerscale"] = 5
figs = catplot(cells.loc[cells.neighborhood100.isin(neigh_list)],X = 'Xcorr',Y='Ycorr',exp = 'array',
               hue = 'neighborhood'+str(k),invert_y=True,size = 1,figsize=8)

In [ ]:
#this plot shows the types of cells (ClusterIDs) in the different niches (0-9)
#font size of graph
SMALL_SIZE = 14
MEDIUM_SIZE = 16
BIGGER_SIZE = 18

#Settings for graph
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [ ]:
col_list = list(sum_cols)
hm = cells.groupby(['neighborhood100'])[col_list].mean()
sns.clustermap(hm, cmap='viridis', standard_scale=0)

In [ ]:
#this plot shows the types of cells (ClusterIDs) in the different niches (0-9)
k_to_plot = k
niche_clusters = (k_centroids[k_to_plot])
tissue_avgs = values.mean(axis = 0)
fc = np.log2(((niche_clusters+tissue_avgs)/(niche_clusters+tissue_avgs).sum(axis = 1, keepdims = True))/tissue_avgs)
fc = pd.DataFrame(fc,columns = sum_cols)
s=sns.clustermap(fc, vmin =-3,vmax = 3,cmap = 'bwr', figsize=(15,15))
#s.savefig(save_path+"celltypes_perniche_"+"_"+str(k)+".png", dpi=600)

In [ ]:
n_conversion_30 = {
    0: 'Follicle',
    1: 'Secretory Epithelial',
    2: 'Smooth Muscle',
    3: 'Plasma Cell Enriched',
    4: 'CD8+ T Enriched IEL',
    5: 'Adaptive Immune Enriched',
    6: 'Plasma Cell Enriched',
    7: 'Mature Epithelial',
    8: 'Stroma',
    9: 'CD66+ Mature Epithelial',
   
    10: 'Stroma',
    11: 'Follicle',
    12: 'CD8+ T Enriched IEL',
    13: 'Smooth Muscle',
    14: 'Smooth Muscle',
    15: 'Innate Immune Enriched',
    16: 'Adaptive Immune Enriched',
    17: 'Secretory Epithelial',
    18: 'Secretory Epithelial',
    19: 'Plasma Cell Enriched',
    

}
cells['Community']=cells['neighborhood100'].map(n_conversion_30)
cells['Community'].unique()

In [ ]:
#modify figure size aesthetics for each neighborhood
plt.rcParams["legend.markerscale"] = 15
figs = catplot(cells,X = 'Xcorr',Y='Ycorr',exp = 'array',
               hue = 'Community',invert_y=True,size = 1,figsize=15)

In [ ]:
pal_color = {
'CD44hi Tumor': 'tan',
 'Tumor': 'gray',
 'Ki67+ Tumor': 'beige',
 'Ki67+ TYRP1hi Tumor': 'brown',
 'Ki67+ CD71hi Tumor': 'orange',
 'APC Enriched Tumor': 'green',
 'Inflamed Tumor': 'magenta',
 'Vascularized Immune Infiltrate': 'black',
 'Unproductive T cell Tumor Interface': 'red',
 'Immune Infiltrate': 'skyblue',
 'TYRP1hi Tumor': 'olive',
 'Productive T cell Tumor Interface': 'blue',
 'Neutrophil Enriched Immune Infiltrate': 'yellow',
 'Neutrophil Enriched': 'gold'
}

pal_color_cells = {
  'EGFR+ Epithelial cell':'blue',
 'Smooth muscle cell':'red',
 'Epithelial cell':'yellow',
  'Nerve cell':'magenta',
  'Macrophage':'orange',  
    'CD4+ Treg':'green',
    'Endothelial cell':'brown', 
     'CD8+ T cell':'black',
 'Intestinal Epithelial cell':'gray',
  'Neutrophil':'skyblue',
 'Parietal cell':'fuchsia',
   'Plasma cell':'gold',  
     'NK cell':'plum',
   'CD4+ T cell':'yellowgreen',
   'CK7+ Epithelial cell':'tan',
   'Neck cell':'navy',
  'Neuroendocrine cell':'bisque',   
 'Chief cell':'goldenrod',
  'Goblet cell':'blueviolet',
  'Foveolar cell':'darkorange',   
 'Stromal cell':'teal',
 'PDPN+ Stromal cell':'olive',
 'p63+ EGFR+ Epithelial cell':'dimgray',
 'Gastric mucouse secreting cell':'indigo',
 'Biglycan+ Stromal cell':'lightcoral',
 'Dendritic cell':'cyan',
 'Paneth cell':'ivory',
 'B cell':'beige',
 'Squamous Epithelial cell':'darkblue',
 'Immune unknown cell':'lightcyan',
 'p63+ Epithelial cell':'royalblue',
}

In [ ]:
n_conversion_c = {
'Plasma Cell Enriched':'Immune',
 'Mature Epithelial':'Epithelial',
 'Innate Immune Enriched':'Immune',
 'Stroma':'Stroma',
 'Follicle':'Immune',
 'Adaptive Immune Enriched':'Immune',
 'Smooth Muscle':'Smooth Muscle',
 'Secretory Epithelial':'Epithelial',
 'CD66+ Mature Epithelial':'Epithelial',
 'CD8+ T Enriched IEL':'Epithelial'
    

}
cells['Major Community']=cells['Community'].map(n_conversion_c)
cells['Major Community'].unique()

In [ ]:
com_pal = {'Immune':'blue', 'Epithelial':'green', 'Stroma':'orange', 'Smooth Muscle':'red'}

In [ ]:
conver = {'Immune':'Mucosa', 'Epithelial':'Mucosa', 'Stroma':'Subucosa', 'Smooth Muscle':'Muscle'}
cells['Tissue Segment']=cells['Major Community'].map(conver)
cells['Tissue Segment'].unique()

In [ ]:
#modify figure size aesthetics for each neighborhood
plt.rcParams["legend.markerscale"] = 15
figs = catplot(cells,X = 'Xcorr',Y='Ycorr',exp = 'array',
               hue = 'Tissue Segment',invert_y=True,size = 1,figsize=15)

In [ ]:
#modify figure size aesthetics for each neighborhood
plt.rcParams["legend.markerscale"] = 15
figs = catplot(cells,X = 'Xcorr',Y='Ycorr',exp = 'array',
               hue = 'Major Community',invert_y=True,size = 1,figsize=15, palette=com_pal)

In [ ]:
cell_list = list(cells['Cell Type'].unique())
neigh_list = list(cells.Community.unique())
color_list=list(pal_color_cells.values())
dict_cell = dict(zip(cell_list, color_list))
dict_neigh = dict(zip(neigh_list, color_list))
dict_neigh

In [ ]:
list(cells['Major Community'].unique())

In [ ]:
dict_comm = {'Plasma Cell Enriched': 'yellow',
 'Mature Epithelial': 'magenta',
 'Innate Immune Enriched': 'brown',
 'Stroma': 'gray',
 'Follicle': 'blue',
 'Adaptive Immune Enriched': 'orange',
 'Smooth Muscle': 'red',
 'Secretory Epithelial': 'purple',
 'CD66+ Mature Epithelial': 'teal',
 'CD8+ T Enriched IEL': 'green'}

In [ ]:
dict_neigh2 = {'Plasma Cell Enriched': 'yellow',
 'Mature Epithelial': 'magenta',
 'Innate Immune Enriched': 'brown',
 'Stroma': 'gray',
 'Follicle': 'blue',
 'Adaptive Immune Enriched': 'orange',
 'Smooth Muscle': 'red',
 'Secretory Epithelial': 'purple',
 'CD66+ Mature Epithelial': 'teal',
 'CD8+ T Enriched IEL': 'green'}

In [ ]:
#modify figure size aesthetics for each neighborhood
plt.rcParams["legend.markerscale"] = 15
figs_n = catplot(cells,X = 'Xcorr',Y='Ycorr',exp = 'array',
               hue = 'Community',invert_y=True,size = 1,figsize=8, palette=dict_neigh2)

In [ ]:
cell_map = {
    'NK': 'blue',
 'Enterocyte': 'magenta',
 'MUC1+ Enterocyte': 'yellow',
 'TA': 'skyblue',
 'CD66+ Enterocyte': 'darkorange',
 'Paneth': 'green',
 'Smooth muscle': 'red',
 'Cycling TA': 'tan',
 'M1 Macrophage': 'gray',
 'Goblet': 'indigo',
 'Neuroendocrine': 'yellowgreen',
 'CD57+ Enterocyte': 'lightsalmon',
 'Lymphatic': 'plum',
 'CD8+ T': 'gold',
 'DC': 'blueviolet',
 'M2 Macrophage': 'bisque',
 'B': 'navy',
 'Neutrophil': 'goldenrod',
 'Endothelial': 'black',
 'Plasma': 'yellow',
 'CD4+ T cell': 'brown',
 'Stroma': 'dimgray',
 'Nerve': 'olive',
 'ICC': 'teal',
 'CD7+ Immune': 'lightcoral'}


neigh_map = {
 'Transit Amplifying Zone': 'darkblue',
 'Microvasculature': 'black',
 'Adatpive Immune Enriched': 'orange',
 'Glandular Epithelial': 'darkorange',
 'CD66+ Mature Epithelial': 'firebrick',
 'Stroma & Innate Immune': 'brown',
 'CD8+ T Enriched IEL': 'green',
 'Mature Epithelial': 'magenta',
 'Innate Immune Enriched': 'skyblue',
 'Outer Follicle': 'navy',
 'Plasma Cell Enriched': 'yellow',
 'Innervated Stroma': 'blueviolet',
 'Stroma': 'gray',
 'Macrovasculature': 'gold',
 'Secretory Epithelial': 'yellowgreen',
 'Smooth Muscle': 'red',
 'Innervated Smooth Muscle': 'lightgreen',
 'Inner Follicle': 'blue',
 'Smooth Muscle & Innate Immune': 'tan',
 'Paneth Enriched': 'lightblue'}

comm_map = {'Plasma Cell Enriched': 'yellow',
 'Mature Epithelial': 'magenta',
 'Innate Immune Enriched': 'brown',
 'Stroma': 'gray',
 'Follicle': 'blue',
 'Adaptive Immune Enriched': 'orange',
 'Smooth Muscle': 'red',
 'Secretory Epithelial': 'purple',
 'CD66+ Mature Epithelial': 'teal',
 'CD8+ T Enriched IEL': 'green'}

In [ ]:
neigh_data1 = pd.DataFrame({
    'Neighborhood':list(neigh_map.keys()),
    'color':list(neigh_map.values())
})
neigh_data1.set_index(keys='Neighborhood',inplace=True)
neigh_data1

In [ ]:
neigh_data = pd.DataFrame({
    'Neighborhood':list(comm_map.keys()),
    'color':list(comm_map.values())
})
neigh_data.set_index(keys='Neighborhood',inplace=True)
neigh_data

In [ ]:
save_path = 'G:/HuBMAP/B008_12/Neighborhoods/'

In [ ]:
#this plot shows the types of cells (ClusterIDs) in the different niches (0-9)
sum_cols = cells['Neighborhood'].unique()
values = cells[sum_cols].values
sum_cols = cells['Neighborhood'].unique()
#cells.rename(columns=n_conversion_cell, inplace=True)
cells[sum_cols]

#Find cell frequency within the neighborhoods
cell_list = list(sum_cols)
cell_list.append('Community')
subset = cells[cell_list]
niche_sub = subset.groupby('Community').sum()
niche_df = niche_sub.apply(lambda x: x/x.sum() * 10, axis=1)
neigh_clusters = niche_df.to_numpy()

tissue_avgs = values.mean(axis = 0)
fc_2 = np.log2(((neigh_clusters+tissue_avgs)/(neigh_clusters+tissue_avgs).sum(axis = 1, keepdims = True))/tissue_avgs)
fc_2 = pd.DataFrame(fc_2,columns = sum_cols)
fc_2.set_index(niche_df.index, inplace=True)
s=sns.clustermap(fc_2, vmin =-3,vmax = 3,cmap = 'bwr', figsize=(12,8), row_colors=[neigh_data.reindex(fc_2.index)['color']],\
                cbar_pos=(0.01,0.07,0.03,0.1))

s.ax_row_dendrogram.set_visible(False)
s.ax_col_dendrogram.set_visible(False)
s.ax_heatmap.set_ylabel("", labelpad=25)
s.ax_heatmap.tick_params(axis='y', pad=25)
s.ax_heatmap.yaxis.set_ticks_position("left")

s.savefig(save_path+"community_heatmap_20_100nn.png", dpi=600)
print(save_path)

In [ ]:
for n,f in enumerate(figs_cs):
    f.savefig(save_path+'HuBMAP_cellmap{}_hr.png'.format(n), dpi=600)
for n,f in enumerate(figs_ns):
    f.savefig(save_path+'HuBMAP_commmap{}_hr.png'.format(n), dpi=600)

In [ ]:
cells_out = cells.drop(columns=sum_cols)
cells_out.columns

In [ ]:
drop_list = [ 'neighborhood100','Unnamed: 0',]
cells_out.drop(columns=drop_list, inplace=True)
cells_out.columns

In [ ]:
# df_dict = {}
# for tissue in list(cells_out.Tissue_location.unique()):
#     df_dict[tissue] = cells_out.loc[cells_out.Tissue_location==tissue]
#     df_dict[tissue].to_csv(save_path+tissue+'_'+'4_12_HuBMAP_Neighborhoods.csv')

In [ ]:
cells_out.to_csv(save_path+'5_2_HuBMAP_Communities.csv')